# SpikeInterface Processing Pipeline for OpenEphys Neuropixels 2 recordings
### Jake Swann, 2024

##### This is a notebook which takes a spreadsheet as input with information on NP2 OpenEphys recordings, and sorts all unsorted recordings in a loop. It will concatenate all recordings made on each unique day, and sort them all together, to be split apart afterwards
##### Each path in the spreadsheet should be to a folder containing all openephys recordings in a given day
##### Required spreadsheet columns are: `trial_name, path (to trial recording), probe_type ('NP2_openephys'), num_channels (384), include ('Y')`
##### The script loads them as a [SpikeInterface](https://github.com/SpikeInterface) object & attaches probe geometry, spike sorts using [Kilosort 4](https://github.com/MouseLand/Kilosort), and allows curation of the output in the [phy](https://github.com/cortex-lab/phy/) template-gui
##### **N.B.** This requires a Python 3.8 environment with SpikeInterface v0.101 + installed
---

In [1]:
import numpy as np
import pandas as pd
import spikeinterface as si
import spikeinterface.extractors as se
from pyscan.session_utils import gs_to_df
from pyscan.np2_utils.np2_preprocessing import sort_np2

# Load sheet
sheet = gs_to_df('https://docs.google.com/spreadsheets/d/1_Xs5i-rHNTywV-WuQ8-TZliSjTxQCCqGWOD2AL_LIq0/edit#gid=0')
path_to_data = '/home/isabella/Documents/isabella/jake/recording_data/'
sorting_suffix = 'sorting_ks4'
sheet['path'] = path_to_data + sheet['path']

# Select only NP2 data to be sorted
sheet_inc = sheet[sheet['Include'] == 'Y']
sheet_inc = sheet_inc[sheet_inc['probe_type'] == 'NP2_openephys']

trial_list = sheet_inc['trial_name'].to_list()
session_list = np.unique([f"{i.split('_')[0]}_{i.split('_')[1]}" for i in trial_list])
recording_list = [[] for _ in range(len(session_list))]

# Collect recordings into sessions and preprocess
for i, session in enumerate(session_list):
    for trial in trial_list:
        if session in trial:
            base_folder = sheet_inc[sheet_inc['trial_name'] == trial]['path'].tolist()[0]
            num_channels = int(sheet_inc[sheet_inc['trial_name'] == trial]['num_channels'].tolist()[0])
            electrode_type = sheet_inc[sheet_inc['trial_name'] == trial]['probe_type'].tolist()[0]
            print(f"{base_folder}/{trial}")
            recording = se.read_openephys(folder_path=f"{base_folder}/{trial}", stream_id = '0')
            recording_list[i].append([recording,
                                     trial, 
                                     base_folder, 
                                     electrode_type])

# Concatenate over a single session and sort
for recording in recording_list:
    session = pd.DataFrame(recording)
    base_folder = session.iloc[0,2]

    # Concatenate recordings save to .dat
    recordings_concat = si.concatenate_recordings(session.iloc[:,0].to_list())
    si.write_binary_recording(recordings_concat, f'{base_folder}/concat.dat')
    print(f'Concatenated recording saved to {base_folder}/concat.dat')

    print(f'Sorting {recordings_concat}')
    sorting = sort_np2(recording = recordings_concat, 
			recording_name = session.iloc[0,1], 
			base_folder = session.iloc[0,2],
			sorting_suffix = sorting_suffix)
    session.to_csv(f'{session.iloc[0,2]}/{session.iloc[0,1][:6]}_{sorting_suffix}/session.csv') #save session trial info to .csv

/home/isabella/Documents/isabella/jake/recording_data/r1503/2024-03-15/240315_r1503_open-field_1


write_binary_recording:   0%|          | 0/621 [00:00<?, ?it/s]

Concatenated recording saved to /home/isabella/Documents/isabella/jake/recording_data/r1503/2024-03-15/concat.dat
Sorting ConcatenateSegmentRecording: 384 channels - 30.0kHz - 1 segments - 18,613,123 samples 
                             620.44s (10.34 minutes) - int16 dtype - 13.31 GiB
Loading recording with SpikeInterface...
number of samples: 18613123
number of channels: 384
numbef of segments: 1
sampling rate: 30000.0
dtype: int16
Preprocessing filters computed in  1.41s; total  1.41s

computing drift
Re-computing universal templates from data.


100%|██████████| 311/311 [03:37<00:00,  1.43it/s]


drift computed in  220.45s; total  221.86s

Extracting spikes using templates
Re-computing universal templates from data.


...

100%|██████████| 311/311 [01:18<00:00,  3.95it/s]


1017214 spikes extracted in  78.96s; total  553.51s

Final clustering


100%|██████████| 96/96 [00:40<00:00,  2.37it/s]


375 clusters found, in  40.59s; total  594.11s

Merging clusters
350 units found, in  1.64s; total  595.75s

Saving to phy and computing refractory periods
170 units found with good refractory periods

Total runtime: 597.03s = 00:09:57 h:m:s
kilosort4 run time 597.37s
Recording sorted!
 KS4 found 350 units

Sorting saved to /home/isabella/Documents/isabella/jake/recording_data/r1503/2024-03-15/240315_sorting_ks4/sort



/data/isabella/jake/ephys_analysis/spikeinterface/src/spikeinterface/core/basesorting.py:239: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")


In [18]:
session.iloc[0,1]

'240315_r1503_open-field_1'

#### Unused Code

In [ ]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.widgets as sw

recording_path = '/data/isabella/jake/recording_data/NP2 data/2024-03-15/test/2024-03-15_13-05-49'
sorting_path = '/data/isabella/jake/recording_data/NP2 data/2024-03-15/test/kilosort4'

recording = se.read_openephys(folder_path=recording_path, stream_id = '0')
sorting = se.read_phy(sorting_path, exclude_cluster_groups=['noise', 'mua'])


import spikeinterface.postprocessing as sp
sorting_analyzer = si.create_sorting_analyzer(sorting=sorting, recording=recording)
sorting_analyzer.compute('random_spikes')
sorting_analyzer.compute('waveforms')
sorting_analyzer.compute_one_extension('templates')
si.postprocessing.compute_template_metrics(sorting_analyzer)
unit_locations = sorting_analyzer.compute(input="unit_locations", method="monopolar_triangulation")

sw.plot_rasters(sorting, time_range=[0, 10])



Loading recording with SpikeInterface...
number of samples: 18613123
number of channels: 384
numbef of segments: 1
sampling rate: 30000.0
dtype: int16
Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
Using GPU for PyTorch computations. Specify `device` to change this.


TypeError: string indices must be integers